# 결정 트리 실습

사용자 행동 인식 데이터 세트

[원본 데이터셋]
https://archive.ics.uci.edu/ml/datasets/human+activity+recognition+using+smartphones

데이터 세트 정보:

실험은 19-48세 사이의 30명의 지원자 그룹으로 수행되었습니다. 각자 스마트폰(Samsung Galaxy S II)을 허리에 차고 6가지 활동(
1:WALKING, 
2:WALKING_UPSTAIRS, 
3:WALKING_DOWNSTAIRS, 4:SITTING, 5:STANDING, 6:LAYING)을 수행했습니다. 내장된 가속도계와 자이로스코프를 사용하여 50Hz의 일정한 속도로 3축 선형 가속도와 3축 각속도를 캡처했습니다. 실험은 데이터에 수동으로 레이블을 지정하기 위해 비디오로 녹화되었습니다. 얻은 데이터 세트는 훈련 데이터 생성을 위해 70%, 테스트 데이터 생성을 위해 30%가 선택된 두 세트로 무작위로 분할되었습니다.

센서 신호(가속도계 및 자이로스코프)는 노이즈 필터를 적용하여 사전 처리된 다음 2.56초 및 50% 중첩(128 판독/창)의 고정 너비 슬라이딩 창에서 샘플링되었습니다. 중력 및 신체 운동 성분을 갖는 센서 가속도 신호는 Butterworth 저역 통과 필터를 사용하여 신체 가속도와 중력으로 분리되었습니다. 중력은 저주파 성분만 있다고 가정하므로 차단 주파수가 0.3Hz인 필터를 사용했습니다. 각 창에서 시간 및 주파수 영역에서 변수를 계산하여 특징 벡터를 얻었습니다.

이 데이터 세트에 대한 자세한 내용은 README.txt 파일을 확인하십시오.

참가자 중 한 명과 함께 녹화된 6가지 활동의 예를 포함하는 실험 비디오는 다음 링크에서 볼 수 있습니다.

https://www.youtube.com/watch?v=XOEN9W05_4A]


**Mission**

수집된 데이터를 기반으로 결정트리를 이용해 어떤 행동을 하고 있는지 예측해 봅시다.

In [ ]:
# ## 코랩을 사용할 때
# #드라이브 마운트
# from google.colab import drive
# drive.mount('/content/drive')

# #현재 작업 위치 이동
# #띄어쓰기에 \붙일 것
# %cd /content/drive/Othercomputers/내\ 노트북_before/Devpy/13.머신러닝

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

## 데이터 로드

1. 피처만 있는 파일을 로드
2. 훈련데이터와 피처를 합쳐 훈련 데이터 프레임 만듬
3. 데스트데이터와 피처를 합쳐 테스트 데이터 프레임 만듬

In [3]:
# 피처이름 로드 
feature_name_df = pd.read_csv('./dataset/human_activity/features_new.txt', header=None,names=['column_name'])
feature_name_df


,column_name
0,tBodyAcc-mean()-X
1,tBodyAcc-mean()-Y
2,tBodyAcc-mean()-Z
3,tBodyAcc-std()-X
4,tBodyAcc-std()-Y
...,...
556,"angle(tBodyGyroMean,gravityMean)"
557,"angle(tBodyGyroJerkMean,gravityMean)"
558,"angle(X,gravityMean)"
559,"angle(Y,gravityMean)"


In [4]:
# 피처이름을 리스트로
feature_name = feature_name_df['column_name'].values.tolist()
# 데이터 프레임에서 리스트로 변환

In [5]:
# 학습 피처 데이터 셋과 테스트 피처 데이터을 DataFrame으로 로딩. 컬럼명은 feature_name 적용

X_train = pd.read_csv('./dataset/human_activity/train/X_train.txt', sep= '\s+', names= feature_name)
X_test = pd.read_csv('./dataset/human_activity/test/X_test.txt', sep = '\s+' ,names = feature_name)


pandas.core.frame.DataFrame

In [6]:
X_train.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)"
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.074323,-0.298676,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,0.158075,-0.595051,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,0.414503,-0.390748,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,0.404573,-0.117290,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,0.087753,-0.351471,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892


In [7]:
# 학습 레이블과 테스트 레이블 데이터을 DataFrame으로 로딩하고 컬럼명은 action으로 부여

y_train = pd.read_csv('./dataset/human_activity/train/y_train.txt', sep = '\s+', header=None, names=['action'])
y_test = pd.read_csv('./dataset/human_activity/test/y_test.txt', sep = '\s+', header=None,names = ['action'])

In [8]:
y_train

,action
0,5
1,5
2,5
3,5
4,5
...,...
7347,2
7348,2
7349,2
7350,2


In [9]:
#action 피처의 class별 갯수
y_train['action'].value_counts()

6    1407
5    1374
4    1286
1    1226
2    1073
3     986
Name: action, dtype: int64

In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# DesisionTree분류 클래스 사용(randomstate 156)

dt_clf = DecisionTreeClassifier(random_state=156)


# 훈련

dt_clf.fit(X_train,y_train)

# 예측

pred = dt_clf.predict(X_test)

#평가

accuracy = accuracy_score(y_test, pred)
accuracy

0.8547675602307431

In [11]:
# DecisionTreeClassifier의 하이퍼 파라미터 추출
print('DecisionTreeClassifier 기본 하이퍼 파라미터:\n', dt_clf.get_params())

DecisionTreeClassifier 기본 하이퍼 파라미터:
 {'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'random_state': 156, 'splitter': 'best'}


## 하이퍼파라미터 튜닝1

In [12]:
from sklearn.model_selection import GridSearchCV

#max_depth를 바꿔가며 훈련해보자
#나무의 깊이를 얼마만큼 성장시킬것인가

params = {
    'max_depth' : [ 6, 8 ,10, 12, 16 ,20, 24]
}


In [13]:
# GridSearchCV 사용

grid_dt_clf = GridSearchCV(dt_clf, param_grid = params, cv = 5)

# 훈련

grid_dt_clf.fit(X_train,y_train)



KeyboardInterrupt: 

In [13]:
# GridSearchCV객체의 cv_results_ 속성을 DataFrame으로 생성. 
scores_df = pd.DataFrame(grid_dt_clf.cv_results_)
scores_df


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,1.503574,0.014092,0.005803,0.000740,6,{'max_depth': 6},0.813732,0.872876,0.819728,0.865986,0.881633,0.850791,0.028313,5
1,1.898851,0.031872,0.006067,0.000126,8,{'max_depth': 8},0.820530,0.819850,0.855102,0.868707,0.891156,0.851069,0.027719,3
2,2.234958,0.065019,0.006206,0.000392,10,{'max_depth': 10},0.799456,0.814412,0.863265,0.891156,0.887755,0.851209,0.037707,2
3,2.527102,0.117175,0.006186,0.000717,12,{'max_depth': 12},0.794697,0.813052,0.848980,0.877551,0.886395,0.844135,0.035598,7
4,2.795610,0.207324,0.006202,0.000393,16,{'max_depth': 16},0.799456,0.818491,0.853061,0.887755,0.897959,0.851344,0.038140,1
5,2.852745,0.238080,0.005811,0.000393,20,{'max_depth': 20},0.802855,0.818491,0.856463,0.877551,0.898639,0.850800,0.035718,4
6,2.841369,0.254711,0.006013,0.000626,24,{'max_depth': 24},0.796057,0.818491,0.856463,0.877551,0.898639,0.849440,0.037597,6


In [14]:
#베스트 분류기 선택
print('GridSearchCV 최적 파라미터:', grid_dt_clf.best_params_)
print('GridSearchCV 최고 정확도: {0:.4f}'.format(grid_dt_clf.best_score_))


#예측
estimator = grid_dt_clf.best_estimator_


#평가 
pred = estimator.predict(X_test)
print('테스트 데이터 세트 정확도: {0:.4f}'.format(accuracy_score(y_test,pred)))

GridSearchCV 최적 파라미터: {'max_depth': 16}
GridSearchCV 최고 정확도: 0.8513
테스트 데이터 세트 정확도: 0.8575


```
    하이퍼파라미터 튜닝으로 성능이 향상 되었나요?
```

## 하이퍼파라미터 튜닝2

In [14]:
#하이퍼파라미터
params = {
    'max_depth' : [ 8 , 12, 16 ,20], 
    'min_samples_split' : [16,24],
}


In [15]:
# GridSearchCV 사용
grid_dt_clf = GridSearchCV(dt_clf, param_grid = params, cv = 5)


# 훈련

grid_dt_clf.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=156),
             param_grid={'max_depth': [8, 12, 16, 20],
                         'min_samples_split': [16, 24]})

In [16]:
#베스트 분류기 선택
print('GridSearchCV 최적 파라미터:', grid_dt_clf.best_params_)
print('GridSearchCV 최고 정확도: {0:.4f}'.format(grid_dt_clf.best_score_))


#예측
estimator = grid_dt_clf.best_estimator_


#평가 
pred = estimator.predict(X_test)
print('테스트 데이터 세트 정확도: {0:.4f}'.format(accuracy_score(y_test,pred)))

GridSearchCV 최적 파라미터: {'max_depth': 8, 'min_samples_split': 16}
GridSearchCV 최고 정확도: 0.8549
테스트 데이터 세트 정확도: 0.8717


```
    하이퍼파라미터 튜닝으로 성능이 향상 되었나요?
```

---

**[생활탐구]**


```
방문을 꼭 닫은채 공부하고 있다는 보검이...
정말 보검이는 공부하고 있을까요?
생체 신호 측정기를 착용한 보검이로 부터 아래와 같은 신호를 가져왔습니다. 
지금 현재 보검이는 무엇을 하고 있나요?

[보검이 생체 신호 파일]X_quiz.txt 

```

In [17]:
#신호 읽어서 예측해보기

X_quiz55 = pd.read_csv('./dataset/human_activity/test/X_quiz55.txt', names= feature_name)


In [63]:
X_train

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)"
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.074323,-0.298676,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,0.158075,-0.595051,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,0.414503,-0.390748,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,0.404573,-0.117290,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,0.087753,-0.351471,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7347,0.299665,-0.057193,-0.181233,-0.195387,0.039905,0.077078,-0.282301,0.043616,0.060410,0.210795,...,-0.070157,-0.588433,-0.880324,-0.190437,0.829718,0.206972,-0.425619,-0.791883,0.238604,0.049819
7348,0.273853,-0.007749,-0.147468,-0.235309,0.004816,0.059280,-0.322552,-0.029456,0.080585,0.117440,...,0.165259,-0.390738,-0.680744,0.064907,0.875679,-0.879033,0.400219,-0.771840,0.252676,0.050053
7349,0.273387,-0.017011,-0.045022,-0.218218,-0.103822,0.274533,-0.304515,-0.098913,0.332584,0.043999,...,0.195034,0.025145,-0.304029,0.052806,-0.266724,0.864404,0.701169,-0.779133,0.249145,0.040811
7350,0.289654,-0.018843,-0.158281,-0.219139,-0.111412,0.268893,-0.310487,-0.068200,0.319473,0.101702,...,0.013865,0.063907,-0.344314,-0.101360,0.700740,0.936674,-0.589479,-0.785181,0.246432,0.025339


In [19]:
X_quiz55

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)"
0,0.173769,-0.024869,-0.094733,-0.893862,-0.944204,-0.960824,-0.904192,-0.950321,-0.967834,-0.923737,...,-0.267475,0.199002,-0.324917,-0.089103,0.10531,0.183368,-0.169434,0.751465,-0.484928,-0.505


In [22]:
# 예측

pred = estimator.predict(X_quiz55)
pred


array([6], dtype=int64)

In [ ]:
# 6번 = 눕기